# Quick Model Security Scan

**Interactive scanner** - just paste a HuggingFace model URL and click Scan!

## Setup

Run the cells below in order. The interactive scanner will appear at the bottom.

In [ ]:
# Import required libraries
import os
from uuid import UUID
from model_security_client.api import ModelSecurityAPIClient
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
from datetime import datetime

print("✓ Libraries imported successfully")

In [ ]:
# Configure credentials from environment variables
CLIENT_ID = os.getenv('MODEL_SECURITY_CLIENT_ID', '')
CLIENT_SECRET = os.getenv('MODEL_SECURITY_CLIENT_SECRET', '')
TSG_ID = os.getenv('TSG_ID', '')

# Security Group UUID (replace with yours)
HUGGINGFACE_GROUP_UUID = UUID("ceb2f363-0166-4872-a44e-f92f684f8a09")

# Set in environment for SDK
if CLIENT_ID and CLIENT_SECRET and TSG_ID:
    os.environ['MODEL_SECURITY_CLIENT_ID'] = CLIENT_ID
    os.environ['MODEL_SECURITY_CLIENT_SECRET'] = CLIENT_SECRET
    os.environ['TSG_ID'] = TSG_ID
    
    print("✓ Credentials configured")
    print(f"  Client ID: {CLIENT_ID[:20]}...")
    print(f"  Security Group: {HUGGINGFACE_GROUP_UUID}")
else:
    print("❌ ERROR: Credentials not configured!")
    print("   Set environment variables: MODEL_SECURITY_CLIENT_ID, MODEL_SECURITY_CLIENT_SECRET, TSG_ID")

In [ ]:
# Initialize Model Security Client
client = ModelSecurityAPIClient(
    base_url="https://api.sase.paloaltonetworks.com/aims"
)

print("✓ Model Security client initialized")
print(f"\n✅ Ready to scan!")

---

## Interactive Scanner

Paste a HuggingFace model URL below and click **Scan Model**.

**Example URLs:**
- `https://huggingface.co/openai-community/gpt2`
- `https://huggingface.co/google-bert/bert-base-uncased`
- `https://huggingface.co/microsoft/DialoGPT-medium`
- `https://huggingface.co/ykilcher/totally-harmless-model`

In [ ]:
# Create interactive widgets
model_url_input = widgets.Text(
    value='https://huggingface.co/openai-community/gpt2',
    placeholder='Paste HuggingFace model URL here',
    description='Model URL:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='600px')
)

scan_button = widgets.Button(
    description='🔍 Scan Model',
    button_style='primary',
    tooltip='Click to scan the model',
    icon='search'
)

output_area = widgets.Output()

# Scan function
def on_scan_clicked(b):
    with output_area:
        clear_output()
        
        model_uri = model_url_input.value.strip()
        
        if not model_uri:
            print("❌ Please enter a model URL")
            return
        
        if not model_uri.startswith('https://huggingface.co/'):
            print("⚠️  Warning: URL should start with https://huggingface.co/")
            print(f"   You entered: {model_uri}")
            print("\n   Proceeding anyway...\n")
        
        print("="*80)
        print(f"🔍 SCANNING: {model_uri}")
        print("="*80)
        print(f"\nStarted: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        print("This may take 30-60 seconds...\n")
        
        try:
            result = client.scan(
                security_group_uuid=HUGGINGFACE_GROUP_UUID,
                model_uri=model_uri
            )
            
            print("="*80)
            print("✅ SCAN COMPLETE")
            print("="*80)
            
            # Determine outcome color
            outcome_str = str(result.eval_outcome)
            if 'ALLOWED' in outcome_str:
                outcome_emoji = "✅"
                outcome_color = "#22c55e"  # green
                border_color = "#22c55e"
            elif 'BLOCKED' in outcome_str:
                outcome_emoji = "🛑"
                outcome_color = "#ef4444"  # red
                border_color = "#ef4444"
            else:
                outcome_emoji = "⚠️"
                outcome_color = "#f59e0b"  # orange
                border_color = "#f59e0b"
            
            display(HTML(f"""
                <div style="border: 2px solid {border_color}; padding: 20px; border-radius: 10px; margin: 10px 0; background-color: rgba(128,128,128,0.1);">
                    <h3 style="margin-top: 0;">{outcome_emoji} Outcome: <span style="color: {outcome_color}; font-weight: bold;">{outcome_str}</span></h3>
                    <p><strong>Scan ID:</strong> <code style="background-color: rgba(128,128,128,0.2); padding: 2px 6px; border-radius: 3px;">{result.uuid}</code></p>
                    <p><strong>Timestamp:</strong> {result.created_at}</p>
                </div>
            """))
            
            if result.eval_summary:
                total = result.eval_summary.total_rules
                passed = result.eval_summary.rules_passed
                failed = result.eval_summary.rules_failed
                pass_rate = (passed / total * 100) if total > 0 else 0
                
                display(HTML(f"""
                    <div style="border: 1px solid rgba(128,128,128,0.3); padding: 15px; border-radius: 8px; margin: 10px 0; background-color: rgba(128,128,128,0.1);">
                        <h4 style="margin-top: 0;">📊 Rules Summary</h4>
                        <ul style="list-style: none; padding-left: 0;">
                            <li>✅ <strong>Passed:</strong> {passed} / {total} ({pass_rate:.1f}%)</li>
                            <li>❌ <strong>Failed:</strong> {failed} / {total}</li>
                        </ul>
                    </div>
                """))
            
            # Additional details
            print(f"\n📁 Files Scanned: {result.total_files_scanned if hasattr(result, 'total_files_scanned') else 'N/A'}")
            print(f"🔧 Scanner Version: {result.scanner_version if hasattr(result, 'scanner_version') else 'N/A'}")
            
            if hasattr(result, 'model_formats') and result.model_formats:
                print(f"\n📦 Model Formats Detected:")
                for fmt in result.model_formats:
                    print(f"   • {fmt}")
            
            print("\n" + "="*80)
            print("💡 VIEW DETAILED RESULTS")
            print("="*80)
            print("For detailed violation information and remediation steps:")
            print("")
            print("1. Go to: https://strata.paloaltonetworks.com")
            print("2. Navigate to: Insights → Prisma AIRS → Model Security → Scans")
            print(f"3. Search for Scan ID: {result.uuid}")
            
        except Exception as e:
            print("="*80)
            print("❌ SCAN FAILED")
            print("="*80)
            print(f"\nError: {str(e)}")
            print("\nPossible issues:")
            print("  • Invalid model URL")
            print("  • Model not accessible")
            print("  • Network/API error")
            print("  • Invalid credentials")

# Attach click handler
scan_button.on_click(on_scan_clicked)

# Display the interface
display(widgets.VBox([
    widgets.HTML('<h3>🔍 Quick Model Scanner</h3>'),
    model_url_input,
    scan_button,
    widgets.HTML('<hr>'),
    output_area
]))

print("\n✅ Interactive scanner ready! Enter a URL above and click 'Scan Model'.")

---

## Quick Reference

### Popular Models to Test

**Safe models:**
- `https://huggingface.co/openai-community/gpt2`
- `https://huggingface.co/google-bert/bert-base-uncased`
- `https://huggingface.co/microsoft/DialoGPT-medium`

**Known vulnerable model:**
- `https://huggingface.co/ykilcher/totally-harmless-model` (contains known backdoor)

### Understanding Results

- **✅ ALLOWED** - Model passed all security rules
- **🛑 BLOCKED** - Model failed one or more security rules
- **⚠️ WARNING** - Model has warnings but not blocked

### Next Steps

1. View detailed results in [Strata Cloud Manager](https://strata.paloaltonetworks.com)
2. Review failed rules and remediation steps
3. Adjust security group rules if needed
4. Scan your own models (local paths, S3, Azure also supported)